In [ ]:
import streamlit as st

class CCTVQuotationModel:
    def __init__(self):
        self.prices = {
            'HD': {'Normal 1080': 1800, 'Smart Hybrid': 2700, 'ColorVu': 3700},
            'IP': {
                'Normal 1080': 4400, 'Smart Hybrid 2MP': 5500, 'Smart Hybrid 4MP': 7500,
                'ColorVu 2MP': 7400, 'ColorVu 4MP': 9000
            },
            'coaxial_cable_100m': 3500, 'ethernet_cable_100m': 3000,
            'bnc_connector': 30, 'rj45_connector': 20, 'power_jack': 30,
            '500gb_hard_disk': 3000, '1tb_hard_disk': 6500, '2tb_hard_disk': 7500,
            'adaptor_box': 200
        }
        self.power_supply_prices = {4: 1800, 8: 2500, 16: 3500}
        self.dvr_nvr_prices = {'HD': {4: 5500, 8: 6500, 16: 11000}, 'IP': {4: 10200, 8: 15500, 16: 28500}}
        self.labour_rate = 0.20

    def generate_quotation(self, category, camera_selection):
        total_cameras = sum(camera_selection.values())
        if 1 <= total_cameras <= 4:
            num_channels = 4
            hard_disk = '500gb_hard_disk'
        elif 5 <= total_cameras <= 8:
            num_channels = 8
            hard_disk = '1tb_hard_disk'
        elif 9 <= total_cameras <= 16:
            num_channels = 16
            hard_disk = '2tb_hard_disk'
        else:
            raise ValueError("Number of cameras must be between 1 and 16.")
        
        camera_cost = 0
        camera_breakdown = {}
        for model, quantity in camera_selection.items():
            if quantity > 0:
                unit_price = self.prices[category].get(model, 0)
                total_price = quantity * unit_price
                camera_cost += total_price
                camera_breakdown[f'{quantity} x {model} @ {unit_price} each'] = total_price

        if category == 'HD':
            bnc_quantity = total_cameras * 2
            power_jack_quantity = total_cameras
            cable_price = self.prices['coaxial_cable_100m']
            cable_quantity = 1 if num_channels == 4 else (2 if num_channels == 8 else 4)
            power_supply_price = self.power_supply_prices.get(num_channels, 0)
            dvr_nvr_price = self.dvr_nvr_prices[category].get(num_channels, 0)
            total_item_cost = (
                dvr_nvr_price + camera_cost + cable_quantity * cable_price + power_supply_price +
                bnc_quantity * self.prices['bnc_connector'] + power_jack_quantity * self.prices['power_jack'] +
                self.prices[hard_disk] + total_cameras * self.prices['adaptor_box']
            )
            cost_breakdown = {
                f'{num_channels} Channel DVR': dvr_nvr_price, 'Cameras': camera_breakdown,
                f'Coaxial Cable ({cable_quantity * 100}m)': cable_quantity * cable_price,
                f'{num_channels} Channel Power Supply': power_supply_price,
                'BNC Connectors': bnc_quantity * self.prices['bnc_connector'],
                'Power Jacks': power_jack_quantity * self.prices['power_jack'],
                hard_disk.replace('_', ' ').title(): self.prices[hard_disk],
                'Adaptor Box': total_cameras * self.prices['adaptor_box']
            }
        else:
            cable_price = self.prices['ethernet_cable_100m']
            cable_quantity = 1 if num_channels == 4 else (2 if num_channels == 8 else 4)
            rj45_quantity = total_cameras * 2
            dvr_nvr_price = self.dvr_nvr_prices[category].get(num_channels, 0)
            total_item_cost = (
                dvr_nvr_price + camera_cost + cable_quantity * cable_price +
                rj45_quantity * self.prices['rj45_connector'] + self.prices[hard_disk] +
                total_cameras * self.prices['adaptor_box']
            )
            cost_breakdown = {
                f'{num_channels} Channel NVR': dvr_nvr_price, 'Cameras': camera_breakdown,
                f'Ethernet Cable ({cable_quantity * 100}m)': cable_quantity * cable_price,
                'RJ45 Connectors': rj45_quantity * self.prices['rj45_connector'],
                hard_disk.replace('_', ' ').title(): self.prices[hard_disk],
                'Adaptor Box': total_cameras * self.prices['adaptor_box']
            }

        labour_cost = total_item_cost * self.labour_rate
        total_cost = total_item_cost + labour_cost
        rounded_total_cost = round(total_cost, -3)
        cost_breakdown['Labour and Transportation Charges'] = labour_cost

        return {'cost_breakdown': cost_breakdown, 'total_cost': rounded_total_cost}

# Streamlit UI code
st.title("CCTV Quotation Generator")
st.subheader("Enter the CCTV Setup Details Below")

# User selects camera category
category = st.selectbox("Select Camera Category", ["HD", "IP"])

# Camera selection inputs
camera_selection = {}
if category == 'HD':
    camera_selection['Normal 1080'] = st.number_input("Quantity for HD Normal 1080", min_value=0, step=1)
    camera_selection['Smart Hybrid'] = st.number_input("Quantity for HD Smart Hybrid", min_value=0, step=1)
    camera_selection['ColorVu'] = st.number_input("Quantity for HD ColorVu", min_value=0, step=1)
elif category == 'IP':
    camera_selection['Normal 1080'] = st.number_input("Quantity for IP Normal 1080 (2MP only)", min_value=0, step=1)
    camera_selection['Smart Hybrid 2MP'] = st.number_input("Quantity for IP Smart Hybrid 2MP", min_value=0, step=1)
    camera_selection['Smart Hybrid 4MP'] = st.number_input("Quantity for IP Smart Hybrid 4MP", min_value=0, step=1)
    camera_selection['ColorVu 2MP'] = st.number_input("Quantity for IP ColorVu 2MP", min_value=0, step=1)
    camera_selection['ColorVu 4MP'] = st.number_input("Quantity for IP ColorVu 4MP", min_value=0, step=1)

# Button to generate quotation
if st.button("Generate Quotation"):
    model = CCTVQuotationModel()
    try:
        quotation = model.generate_quotation(category, camera_selection)
        st.subheader("Quotation Breakdown:")
        for item, cost in quotation['cost_breakdown'].items():
            if isinstance(cost, dict):
                st.write(f"{item}:")
                for cam_item, cam_cost in cost.items():
                    st.write(f"  {cam_item}: KES {cam_cost}")
            else:
                st.write(f"{item}: KES {cost}")
        st.write(f"\n**Total Quotation Amount (rounded): KES {quotation['total_cost']}**")
    except ValueError as e:
        st.error(f"Error: {e}")

In [2]:
import streamlit as st

class CCTVQuotationModel:
    def __init__(self):
        self.prices = {
            'HD': {'Normal 1080': 1800, 'Smart Hybrid': 2700, 'ColorVu': 3700},
            'IP': {
                'Normal 1080': 4400, 'Smart Hybrid 2MP': 5500, 'Smart Hybrid 4MP': 7500,
                'ColorVu 2MP': 7400, 'ColorVu 4MP': 9000
            },
            'coaxial_cable_100m': 3500, 'ethernet_cable_100m': 3000,
            'bnc_connector': 30, 'rj45_connector': 20, 'power_jack': 30,
            '500gb_hard_disk': 3000, '1tb_hard_disk': 6500, '2tb_hard_disk': 7500,
            'adaptor_box': 200
        }
        self.power_supply_prices = {4: 1800, 8: 2500, 16: 3500}
        self.dvr_nvr_prices = {'HD': {4: 5500, 8: 6500, 16: 11000}, 'IP': {4: 10200, 8: 15500, 16: 28500}}
        self.labour_rate = 0.20

    def generate_quotation(self, category, camera_selection):
        total_cameras = sum(camera_selection.values())
        if 1 <= total_cameras <= 4:
            num_channels = 4
            hard_disk = '500gb_hard_disk'
        elif 5 <= total_cameras <= 8:
            num_channels = 8
            hard_disk = '1tb_hard_disk'
        elif 9 <= total_cameras <= 16:
            num_channels = 16
            hard_disk = '2tb_hard_disk'
        else:
            raise ValueError("Number of cameras must be between 1 and 16.")
        
        camera_cost = 0
        camera_breakdown = {}
        for model, quantity in camera_selection.items():
            if quantity > 0:
                unit_price = self.prices[category].get(model, 0)
                total_price = quantity * unit_price
                camera_cost += total_price
                camera_breakdown[f'{quantity} x {model} @ {unit_price} each'] = total_price

        if category == 'HD':
            bnc_quantity = total_cameras * 2
            power_jack_quantity = total_cameras
            cable_price = self.prices['coaxial_cable_100m']
            cable_quantity = 1 if num_channels == 4 else (2 if num_channels == 8 else 4)
            power_supply_price = self.power_supply_prices.get(num_channels, 0)
            dvr_nvr_price = self.dvr_nvr_prices[category].get(num_channels, 0)
            total_item_cost = (
                dvr_nvr_price + camera_cost + cable_quantity * cable_price + power_supply_price +
                bnc_quantity * self.prices['bnc_connector'] + power_jack_quantity * self.prices['power_jack'] +
                self.prices[hard_disk] + total_cameras * self.prices['adaptor_box']
            )
            cost_breakdown = {
                f'{num_channels} Channel DVR': dvr_nvr_price, 'Cameras': camera_breakdown,
                f'Coaxial Cable ({cable_quantity * 100}m)': cable_quantity * cable_price,
                f'{num_channels} Channel Power Supply': power_supply_price,
                'BNC Connectors': bnc_quantity * self.prices['bnc_connector'],
                'Power Jacks': power_jack_quantity * self.prices['power_jack'],
                hard_disk.replace('_', ' ').title(): self.prices[hard_disk],
                'Adaptor Box': total_cameras * self.prices['adaptor_box']
            }
        else:
            cable_price = self.prices['ethernet_cable_100m']
            cable_quantity = 1 if num_channels == 4 else (2 if num_channels == 8 else 4)
            rj45_quantity = total_cameras * 2
            dvr_nvr_price = self.dvr_nvr_prices[category].get(num_channels, 0)
            total_item_cost = (
                dvr_nvr_price + camera_cost + cable_quantity * cable_price +
                rj45_quantity * self.prices['rj45_connector'] + self.prices[hard_disk] +
                total_cameras * self.prices['adaptor_box']
            )
            cost_breakdown = {
                f'{num_channels} Channel NVR': dvr_nvr_price, 'Cameras': camera_breakdown,
                f'Ethernet Cable ({cable_quantity * 100}m)': cable_quantity * cable_price,
                'RJ45 Connectors': rj45_quantity * self.prices['rj45_connector'],
                hard_disk.replace('_', ' ').title(): self.prices[hard_disk],
                'Adaptor Box': total_cameras * self.prices['adaptor_box']
            }

        labour_cost = total_item_cost * self.labour_rate
        total_cost = total_item_cost + labour_cost
        rounded_total_cost = round(total_cost, -3)
        cost_breakdown['Labour and Transportation Charges'] = labour_cost

        return {'cost_breakdown': cost_breakdown, 'total_cost': rounded_total_cost}

# Streamlit UI code
st.title("CCTV Quotation Generator")
st.subheader("Enter the CCTV Setup Details Below")

# User selects camera category
category = st.selectbox("Select Camera Category", ["HD", "IP"])

# Camera selection inputs
camera_selection = {}
if category == 'HD':
    camera_selection['Normal 1080'] = st.number_input("Quantity for HD Normal 1080", min_value=0, step=1)
    camera_selection['Smart Hybrid'] = st.number_input("Quantity for HD Smart Hybrid", min_value=0, step=1)
    camera_selection['ColorVu'] = st.number_input("Quantity for HD ColorVu", min_value=0, step=1)
elif category == 'IP':
    camera_selection['Normal 1080'] = st.number_input("Quantity for IP Normal 1080 (2MP only)", min_value=0, step=1)
    camera_selection['Smart Hybrid 2MP'] = st.number_input("Quantity for IP Smart Hybrid 2MP", min_value=0, step=1)
    camera_selection['Smart Hybrid 4MP'] = st.number_input("Quantity for IP Smart Hybrid 4MP", min_value=0, step=1)
    camera_selection['ColorVu 2MP'] = st.number_input("Quantity for IP ColorVu 2MP", min_value=0, step=1)
    camera_selection['ColorVu 4MP'] = st.number_input("Quantity for IP ColorVu 4MP", min_value=0, step=1)

# Button to generate quotation
if st.button("Generate Quotation"):
    model = CCTVQuotationModel()
    try:
        quotation = model.generate_quotation(category, camera_selection)
        st.subheader("Quotation Breakdown:")
        for item, cost in quotation['cost_breakdown'].items():
            if isinstance(cost, dict):
                st.write(f"{item}:")
                for cam_item, cam_cost in cost.items():
                    st.write(f"  {cam_item}: KES {cam_cost}")
            else:
                st.write(f"{item}: KES {cost}")
        st.write(f"\n**Total Quotation Amount (rounded): KES {quotation['total_cost']}**")
    except ValueError as e:
        st.error(f"Error: {e}")

2024-11-15 11:51:55.594 
  command:

    streamlit run /Applications/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-11-15 11:51:55.595 Session state does not function when running a script without `streamlit run`
